# Objective for Part 2

From the downloaded data in Part 1, we will perform feature engineering:
- Using the location coordinates, we divide the taxi locations throughout Singapore into 9 different sectors.
- We sum up the number of taxis based on their sectors.
- We extract the day of week, minute and hour from the timestamp.
- We drop redundant features from the dataset.
- The cleaned data is then exported as csv.

In [1]:
# Step 1: Import pandas
import pandas as pd
import json
import numpy as np

In [2]:
# Step 2: Read the CSV
df = pd.read_csv("taxi.csv", index_col=False)
df.head()

,type,geometry.type,geometry.coordinates,properties.timestamp,properties.taxi_count,properties.api_info.status,time
0,Feature,MultiPoint,"[[103.6267, 1.307992], [103.63226, 1.30884], [...",2018-12-31T23:59:44+08:00,5887,healthy,2019-01-01 00:00:00
1,Feature,MultiPoint,"[[103.63213, 1.31121], [103.63766, 1.30045], [...",2019-01-01T00:04:44+08:00,4001,healthy,2019-01-01 00:05:00
2,Feature,MultiPoint,"[[103.63145, 1.31125], [103.6376, 1.300248], [...",2019-01-01T00:09:44+08:00,5981,healthy,2019-01-01 00:10:00
3,Feature,MultiPoint,"[[103.63132, 1.3216], [103.63314, 1.32474], [1...",2019-01-01T00:14:45+08:00,5461,healthy,2019-01-01 00:15:00
4,Feature,MultiPoint,"[[103.628, 1.31262], [103.63714, 1.29914], [10...",2019-01-01T00:19:45+08:00,5003,healthy,2019-01-01 00:20:00


In [3]:
# Step 3: Take a look at the first row's geometry coordinates
df.loc[0,'geometry.coordinates']

'[[103.6267, 1.307992], [103.63226, 1.30884], [103.6376, 1.300256], [103.63767, 1.30045], [103.64233, 1.3272], [103.64262, 1.31503], [103.652616666667, 1.3172154], [103.66998, 1.32412], [103.67939, 1.32625], [103.68554, 1.34106], [103.6856, 1.340405], [103.688642833333, 1.340839], [103.689112833333, 1.342593], [103.69163, 1.34406], [103.6931, 1.345999], [103.6936, 1.344527], [103.69386, 1.34267], [103.6939, 1.344551], [103.694, 1.36935], [103.694041833333, 1.34023216666667], [103.69427, 1.33496], [103.69448, 1.34395], [103.6949, 1.339654], [103.694952333333, 1.346155], [103.6959465, 1.34455316666667], [103.696101833333, 1.34533483333333], [103.696348666667, 1.337731], [103.69679, 1.35138], [103.69686, 1.3412], [103.6969, 1.343948], [103.69707, 1.33207], [103.697182666667, 1.33966516666667], [103.6972442, 1.34316185], [103.697827166667, 1.33945583333333], [103.697930666667, 1.34119483333333], [103.6980042, 1.33504375], [103.69861, 1.32278], [103.69997, 1.34669], [103.700030166667, 1.340

In [4]:
# Step 4: Turn the values in 'geometry.coordinates' column into actual lists

In [5]:
def convert_coordinates(str_coordinates):
    list_coordinates = json.loads(str_coordinates)
    return list_coordinates


In [6]:
df['geometry.coordinates'] = df['geometry.coordinates'].map(convert_coordinates)

In [7]:
# Step 5: Sort the coordinates into the nine sectors
list_coords = df['geometry.coordinates'].tolist()

In [8]:
def taxi_section(npa, row):
    try:
        count_1 = np.sum((npa[:,0] >= 103.6) & (npa[:,0] < 103.77) & (npa[:,1] >= 1.37) & (npa[:,1] < 1.48))
        count_2 = np.sum((npa[:,0] >= 103.77) & (npa[:,0] < 103.93) & (npa[:,1] >= 1.37) & (npa[:,1] < 1.48))
        count_3 = np.sum((npa[:,0] >= 103.93) & (npa[:,0] < 104.1) & (npa[:,1] >= 1.37) & (npa[:,1] < 1.48))

        count_4 = np.sum((npa[:,0] >= 103.6) & (npa[:,0] < 103.77) & (npa[:,1] >= 1.26) & (npa[:,1] < 1.37))
        count_5 = np.sum((npa[:,0] >= 103.77) & (npa[:,0] < 103.93) & (npa[:,1] >= 1.26) & (npa[:,1] < 1.37))
        count_6 = np.sum((npa[:,0] >= 103.93) & (npa[:,0] < 104.1) & (npa[:,1] >= 1.26) & (npa[:,1] < 1.37))

        count_7 = np.sum((npa[:,0] >= 103.6) & (npa[:,0] < 103.77) & (npa[:,1] >= 1.15) & (npa[:,1] < 1.26))
        count_8 = np.sum((npa[:,0] >= 103.77) & (npa[:,0] < 103.93) & (npa[:,1] >= 1.15) & (npa[:,1] < 1.26))
        count_9 = np.sum((npa[:,0] >= 103.93) & (npa[:,0] < 104.1) & (npa[:,1] >= 1.15) & (npa[:,1] < 1.26))
    
    except IndexError as e:
        print("Index Error for coordinates at row:", row)
        count_1=0
        count_2=0
        count_3=0
        count_4=0
        count_5=0
        count_6=0
        count_7=0
        count_8=0
        count_9=0
    
    return count_1,count_2,count_3,count_4,count_5,count_6,count_7,count_8,count_9
    

In [9]:
sector_1 = []
sector_2 = []
sector_3 = []
sector_4 = []
sector_5 = []
sector_6 = []
sector_7 = []
sector_8 = []
sector_9 = []

row = 0

for list_coord in list_coords:
    npa = np.array(list_coord, dtype=np.float32)
    if (npa.size > 0):
        count_1,count_2,count_3,count_4,count_5,count_6,count_7,count_8,count_9 = taxi_section(npa, row)
    else:
        count_1=0
        count_2=0
        count_3=0
        count_4=0
        count_5=0
        count_6=0
        count_7=0
        count_8=0
        count_9=0
        
    row += 1
    
    sector_1.append(count_1)
    sector_2.append(count_2)
    sector_3.append(count_3)
    sector_4.append(count_4)
    sector_5.append(count_5)
    sector_6.append(count_6)
    sector_7.append(count_7)
    sector_8.append(count_8)
    sector_9.append(count_9)


In [10]:
df['sector_1'] = sector_1
df['sector_2'] = sector_2
df['sector_3'] = sector_3
df['sector_4'] = sector_4
df['sector_5'] = sector_5
df['sector_6'] = sector_6
df['sector_7'] = sector_7
df['sector_8'] = sector_8
df['sector_9'] = sector_9

In [11]:
# Step 6: convert the strings in "time" column into DateTime object
df['time']= pd.to_datetime(df['time'])

In [12]:
# Step 7: Get dayofweek, minute, hour from 'time' column
df['day_of_week'] = df['time'].dt.dayofweek
df['minute'] = df['time'].dt.minute
df['hour'] = df['time'].dt.hour

In [13]:
# Step 8: Select only the necessary columns for the DataFrame
df_clean = df[['time','properties.taxi_count', 'day_of_week','minute','hour','sector_1','sector_2','sector_3',
              'sector_4','sector_5','sector_6','sector_7','sector_8','sector_9']]

In [14]:
df_clean.head()

,time,properties.taxi_count,day_of_week,minute,hour,sector_1,sector_2,sector_3,sector_4,sector_5,sector_6,sector_7,sector_8,sector_9
0,2019-01-01 00:00:00,5887,1,0,0,167,979,81,408,3507,678,0,67,0
1,2019-01-01 00:05:00,4001,1,5,0,102,496,39,251,2589,471,0,53,0
2,2019-01-01 00:10:00,5981,1,10,0,157,1035,69,417,3546,680,0,77,0
3,2019-01-01 00:15:00,5461,1,15,0,158,920,68,375,3225,654,0,61,0
4,2019-01-01 00:20:00,5003,1,20,0,136,900,62,358,2898,596,0,53,0


In [ ]:
# Step 9: Export the DataFrame from Step 8 as CSV
df_clean.to_csv("taxi2.csv", index=False)